In [0]:
%run /Workspace/Users/ndejong@spn.org/Mailchimp_ETL/Production_ETL/02_Silver_prod

In [0]:
import unittest
from unittest.mock import patch, MagicMock
import pandas as pd
import json

class TestMailchimpSilverETL(unittest.TestCase):

    def test_flatten_and_clean(self):
        from __main__ import flatten_and_clean

        data = [
            {
                "email_address": "user@example.com",
                "last_changed": "2024-01-01T12:00:00Z",
                "merge_fields": {"FNAME": "Alice"},
                "location": {"latitude": 50, "longitude": 8},
                "stats": {"opens": 5}
            },
            {
                "email_address": "user@example.com",
                "last_changed": "2023-01-01T12:00:00Z",
                "merge_fields": {"FNAME": "Old Alice"},
            }
        ]
        df = pd.DataFrame(data)
        result = flatten_and_clean(df)

        self.assertEqual(len(result), 1)
        self.assertIn("merge_FNAME", result.columns)
        self.assertEqual(result.iloc[0]["merge_FNAME"], "Alice")

    @patch('__main__.bronze_fs')
    def test_list_mailchimp_lists(self, mock_bronze_fs):
        from __main__ import list_mailchimp_lists

        # Simulate directory paths with proper string names
        mock_path1 = MagicMock()
        mock_path1.is_directory = True
        mock_path1.name = "mailchimp_members/listName=ListX/ingestion_date=2024-03-01"

        mock_path2 = MagicMock()
        mock_path2.is_directory = False
        mock_path2.name = "mailchimp_members/somefile.json"

        mock_bronze_fs.get_paths.return_value = [mock_path1, mock_path2]

        result = list_mailchimp_lists()
        self.assertIn("ListX", result)
        self.assertEqual(len(result), 1)

    @patch('__main__.bronze_fs')
    def test_read_bronze_json_malformed(self, mock_bronze_fs):
        from __main__ import read_bronze_json

        mock_path = MagicMock()
        mock_path.is_directory = False
        mock_path.name = "mailchimp_members/listName=BadList/file.json"

        mock_file_client = MagicMock()
        mock_file_client.download_file.return_value.readall.return_value = b'{"bad_json":'

        mock_bronze_fs.get_paths.return_value = [mock_path]
        mock_bronze_fs.get_file_client.return_value = mock_file_client

        with self.assertLogs("MailchimpSilverETL", level="ERROR") as cm:
            df = read_bronze_json("BadList")
            self.assertTrue(df.empty)
            self.assertIn("JSON parse failure", "\n".join(cm.output))


# Run inside Databricks cell
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestMailchimpSilverETL))
